In [1]:
import geojson
import geopandas as gpd
import random

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors

In [2]:
data = pd.read_csv('./classif/20x20/ml_v4/classif20x20week_callout/csv/lowdens_results.csv')
#gdf = gpd.read_file('./geojsons/milano-grid.geojson')

with open("./geojsons/milano-grid.geojson") as json_file:
    json_data = geojson.load(json_file)


In [3]:
#Generate the geojson with the true data
for i in range(0, len(data)):
    if(data.loc[i].true == 0): #negative label
        json_data.features[int(data.loc[i].cellid)-1].properties = {
                "fill": colors.to_hex([ 0.95,0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.75,
                "cell-type": "no low dens",
                "cellId": int(data.loc[i].cellid)
            }
    else: #true: positive
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                "fill": colors.to_hex([ 0.98,0.98, 0.98, 0.45 ], keep_alpha = True),
                "fill-opacity": 0.75,
                "stroke-width": 2.5,
                "stroke-opacity": 0.75,
                "cell-type": "low dens",
                "cellId": int(data.loc[i].cellid)
            }

In [4]:
with open("./geojsons/5/truth_20co_ml5_lowdens.geojson", "w") as json_file:
    geojson.dump(json_data, json_file)

In [5]:
with open("./geojsons/milano-grid.geojson") as json_file:
    json_data = geojson.load(json_file) #reset json_data for next dump

In [6]:
#Generate the geojson with the predictions
for i in range(0, len(data)):
    cnt=10000+i
    if(data.loc[i].predict == 0): #negative label: yellow
        if(data.loc[i].true == 0): #negative label, correctly classified
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                #"stroke": colors.to_hex([ 0.95, 0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill": colors.to_hex([ 0.95,0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.95,
                "cell-type": "no low dens",
                "cellId": int(data.loc[i].cellid)
            }
        else: #true: positive - incorrect classification
            coords = json_data.features[int(data.loc[i].cellid)-1]["geometry"].coordinates
            line = geojson.LineString([coords[0][0], coords[0][len(coords[0])-3]])
            json_data['features'].append(geojson.Feature(geometry=line, id=cnt, properties={"stroke": colors.to_hex([ 0.95,0.95, 0.95, 0.99 ], keep_alpha = True)}))
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                "fill": colors.to_hex([ 0.95,0.95, 0.0, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.99,
                "cell-type": "low dens",
                "cellId": int(data.loc[i].cellid)
            }
            cnt+=1
    else: #predicted positive: white
        if(data.loc[i].true == 1): #correct
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                #"stroke": colors.to_hex([ 0.95,0.95, 0.95, 0.95 ], keep_alpha = True),
                "fill": colors.to_hex([ 0.95,0.95, 0.95, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.95,
                "cell-type": "low dens",
                "cellId": int(data.loc[i].cellid)
            }
        else: #incorrect classification (true: negative)
            #print(json_data.features[int(data.loc[i].cellid)-1]["geometry"].coordinates)
            coords = json_data.features[int(data.loc[i].cellid)-1]["geometry"].coordinates
            line = geojson.LineString([coords[0][0], coords[0][len(coords[0])-3]])
            json_data['features'].append(geojson.Feature(geometry=line, id=cnt, properties={"stroke": colors.to_hex([ 0.95,0.95, 0.00, 0.99 ], keep_alpha = True)}))
            json_data.features[int(data.loc[i].cellid)-1].properties = {
                "fill": colors.to_hex([ 0.95,0.95, 0.95, 0.95 ], keep_alpha = True),
                "fill-opacity": 0.4,
                "stroke-width": 2.5,
                "stroke-opacity": 0.99,
                "cell-type": "no low dens",
                "cellId": int(data.loc[i].cellid)
            }
            cnt+=1

In [7]:
with open("./geojsons/5/predictions_20co_ml5_lowdens.geojson", "w") as json_file:
    geojson.dump(json_data, json_file)